In [2]:
import pandas as pd

df_train = pd.read_csv("train.csv")
# test = pd.read_csv("test.csv")

print(df_train.shape)
df_train.head()


(18153, 21)


,ID,ASI_category,Temperature,Precipitation,Rainfall,Snowfall,Soil_Temperature,Radiation,Wind_Speed,Wind_Gusts,...,Surface_Pressure,Relative_Humidity,Soil_Moisture,Dew_Point,Sunshine_Duration,Cloud_Cover,Precipitation_Hours,Wind_Direction,Weather_Code,Daylight_Duration
0,19554,Moderate,0.931231,0.000912,0.000912,0.0,0.757673,0.879671,0.179293,0.193029,...,0.538056,55,0.546243,17.564597,53252.08,12.136192,1,176.459082,51,58772.52
1,25205,Moderate,0.566323,0.096715,0.096715,0.0,0.291448,0.008913,0.588384,0.532172,...,0.568475,88,0.557803,5.692134,0.00,91.901341,16,232.433005,61,28143.12
2,771,Poor,0.018033,0.000000,0.000000,0.0,0.000000,0.277340,0.247475,0.189008,...,0.706520,78,0.791908,-25.264420,30213.79,18.859670,0,44.688600,3,34621.43
3,1976,Good,0.717541,0.000000,0.000000,0.0,0.635669,0.796709,0.123737,0.134048,...,0.547500,57,0.473988,5.913865,44627.21,38.759757,0,333.640418,3,59192.17
4,14036,Moderate,0.827170,0.001825,0.001825,0.0,0.743855,0.781282,0.343434,0.391421,...,0.546378,50,0.459538,9.661455,45267.17,60.058955,1,86.996954,51,59956.03


In [3]:
df_train.dtypes

ID                       int64
ASI_category            object
Temperature            float64
Precipitation          float64
Rainfall               float64
Snowfall               float64
Soil_Temperature       float64
Radiation              float64
Wind_Speed             float64
Wind_Gusts             float64
Pressure_MSL           float64
Surface_Pressure       float64
Relative_Humidity        int64
Soil_Moisture          float64
Dew_Point              float64
Sunshine_Duration      float64
Cloud_Cover            float64
Precipitation_Hours      int64
Wind_Direction         float64
Weather_Code             int64
Daylight_Duration      float64
dtype: object

In [4]:
df_train.isnull().sum()

ID                     0
ASI_category           0
Temperature            0
Precipitation          0
Rainfall               0
Snowfall               0
Soil_Temperature       0
Radiation              0
Wind_Speed             0
Wind_Gusts             0
Pressure_MSL           0
Surface_Pressure       0
Relative_Humidity      0
Soil_Moisture          0
Dew_Point              0
Sunshine_Duration      0
Cloud_Cover            0
Precipitation_Hours    0
Wind_Direction         0
Weather_Code           0
Daylight_Duration      0
dtype: int64

In [5]:
# %%
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode target labels
le = LabelEncoder()
y = le.fit_transform(df_train["ASI_category"])   # One-time encoding

# Split features and target
X = df_train.drop(["ID", "ASI_category"], axis=1)

# Split into train/validation sets (stratified for balance)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.05, random_state=42, stratify=y
)

print("Training set:", X_train.shape, "Validation set:", X_val.shape)
print("Classes:", le.classes_)


Training set: (17245, 19) Validation set: (908, 19)
Classes: ['Good' 'Moderate' 'Poor']


In [ ]:
# # %% [Train XGBoost with tuned parameters and evaluate on Train & Validation sets]

# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
# import numpy as np

# # Define tuned XGBoost model
# best_xgb = XGBClassifier(
#     n_estimators=300,
#     learning_rate=0.01, 
#     max_depth=5, 
#     subsample=0.7, 
#     colsample_bytree=0.6, 
#     gamma=1.39, 
#     min_child_weight=3,
#     random_state=42,
#     use_label_encoder=False,
#     eval_metric="mlogloss",
#     n_jobs=-1
# )

# # Train model
# best_xgb.fit(X_train, y_train)

# # Predictions for both train and validation sets
# y_pred_train = best_xgb.predict(X_train)
# y_pred_val = best_xgb.predict(X_val)

# # ✅ Compute metrics for train and validation sets
# metrics = {}

# for label, y_true, y_pred in [
#     ("Train", y_train, y_pred_train),
#     ("Validation", y_val, y_pred_val)
# ]:
#     acc = accuracy_score(y_true, y_pred)
#     f1 = f1_score(y_true, y_pred, average="macro")
#     rmse = np.sqrt(mean_squared_error(y_true, y_pred))
#     metrics[label] = {"Accuracy": acc, "F1": f1, "RMSE": rmse}

# # Display results nicely
# print("✅ Model Evaluation Metrics:")
# for dataset, scores in metrics.items():
#     print(f"\n📊 {dataset} Set:")
#     print(f"Accuracy : {scores['Accuracy']:.4f}")
#     print(f"F1 Score : {scores['F1']:.4f}")
#     print(f"RMSE     : {scores['RMSE']:.4f}")


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:29:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Model Evaluation Metrics:

📊 Train Set:
Accuracy : 0.9429
F1 Score : 0.9199
RMSE     : 0.2460

📊 Validation Set:
Accuracy : 0.9251
F1 Score : 0.9000
RMSE     : 0.2737


In [20]:
# %% [Optuna Hyperparameter Tuning for XGBoost - Clean Version]

import optuna
from optuna.samplers import TPESampler
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np


def objective(trial):
    # --- Define optimized parameter search space ---
    params = {
        # Core parameters
        "n_estimators": trial.suggest_int("n_estimators", 200, 700),
        "eta": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 2.5),

        # Subsampling for regularization
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),

        # Regularization
        "lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),

        # Booster and objective
        "booster": "gbtree",  # fixed to gbtree (no dart)
        "objective": "multi:softprob",
        "eval_metric": "mlogloss",

        # Misc
        "num_class": len(np.unique(y_train)),
        "random_state": 42,
        "verbosity": 0,
        "nthread": -1,
    }

    # --- Split data into sub-train / validation for evaluation ---
    X_sub, X_hold, y_sub, y_hold = train_test_split(
        X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
    )

    dtrain = xgb.DMatrix(X_sub, label=y_sub)
    dvalid = xgb.DMatrix(X_hold, label=y_hold)

    # --- Train model with early stopping ---
    evals = [(dtrain, "train"), (dvalid, "val")]
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1500,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # --- Predict and evaluate ---
    preds = np.argmax(model.predict(dvalid), axis=1)
    f1 = f1_score(y_hold, preds, average="macro")

    return f1


# --- Run Optuna Study ---
study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=200, show_progress_bar=True)

print("\n✅ Best XGBoost Parameters:")
print(study.best_params)
print(f"Best Macro F1 Score: {study.best_value:.4f}")


[I 2025-10-30 21:05:27,875] A new study created in memory with name: no-name-5fdb611c-c1f1-40f6-ad0d-39c1b33e847a


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2025-10-30 21:05:28,807] Trial 0 finished with value: 0.9065685309504796 and parameters: {'n_estimators': 387, 'learning_rate': 0.17254716573280354, 'max_depth': 8, 'min_child_weight': 6, 'gamma': 0.3900466011060913, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'colsample_bylevel': 0.9330880728874675, 'colsample_bynode': 0.8005575058716043, 'reg_lambda': 3.540362888980227, 'reg_alpha': 0.10292247147901223}. Best is trial 0 with value: 0.9065685309504796.
[I 2025-10-30 21:05:29,899] Trial 1 finished with value: 0.9124751549729174 and parameters: {'n_estimators': 685, 'learning_rate': 0.12106896936002161, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.45851127463358454, 'subsample': 0.6521211214797689, 'colsample_bytree': 0.762378215816119, 'colsample_bylevel': 0.7159725093210578, 'colsample_bynode': 0.645614570099021, 'reg_lambda': 3.0592644736118975, 'reg_alpha': 0.6974693032602092}. Best is trial 1 with value: 0.9124751549729174.
[I 2025-10-30 21:05:3

KeyboardInterrupt: 